In [ ]:
import cv2
from ultralytics import YOLO
import time
import os
 
model_path = r"goomba_detector_9000.pt"
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at {model_path}")

model = YOLO(model_path)

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise IOError("Error: Could not open webcam.")

print("Starting webcam feed... Press 'q' to exit.")


log_file_path = r"detection_log.txt"
log_file = open(log_file_path, "a")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break

        results = model(frame, stream=True)

        # Initialize Goomba counter for the current frame
        goomba_count = 0

        # Process detections
        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                conf = box.conf[0]
                cls = box.cls[0]

                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                class_name = model.names[int(cls)]

                # Check if the detected object is a Goomba
                if class_name.lower() == "goomba":
                    goomba_count += 1

                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # Draw label
                label = f"{class_name} {conf:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Log detection
                timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
                log_entry = f"{timestamp} - {class_name} {conf:.2f} at [{x1}, {y1}, {x2}, {y2}]\n"
                log_file.write(log_entry)

        # Display the Goomba count on the frame
        count_label = f"Goombas on Screen: {goomba_count}"
        cv2.putText(frame, count_label, (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Display the frame
        cv2.imshow('Goomba Detection', frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exiting...")
            break

except KeyboardInterrupt:
    print("Interrupted by user.")

finally:
    cap.release()
    cv2.destroyAllWindows()
    log_file.close()
    print("Resources released and log file closed.")
